In [ ]:
import glob
import pandas as pd
import numpy as np 
from astropy.table import Table
from astropy.io import fits
import glob
import seaborn as sns
import matplotlib.pyplot as plt
import time

In [ ]:
files = glob.glob('/Users/vdk/muons2024/20240131muons/muons_LST-1.*')
len(files)
short_files = files[:40000]
run_number = [file_part[50:55] for file_part in short_files]
sorted_run_number = sorted(run_number, key=int)
set_sorted_run = set(sorted_run_number)
subruns_dict = {}
for run in set_sorted_run:
    subruns_dict[run] = []

for run in set_sorted_run:    
    for subrun in short_files:
        if run in subrun:
            subruns_dict[run].append(subrun)

In [ ]:
all_data_df = pd.DataFrame()

# Assuming subruns_dict is a dictionary where each key corresponds to a subrun
# and its value is a list of FITS file paths for that subrun
t1 = time.perf_counter(), time.process_time()
for subrun_key in subruns_dict:
    for fits_file_path in subruns_dict[subrun_key]:
        try:
            # Read the FITS file into an Astropy Table
            dat = Table.read(fits_file_path)
            
            # Convert the Astropy Table to a Pandas DataFrame
            df = dat.to_pandas()
            # Append the DataFrame to the main DataFrame
            all_data_df = pd.concat([df, df_good_data], ignore_index=True)
        except:
            pass
    t2 = time.perf_counter(), time.process_time()

print(f" Real time: {t2[0] - t1[0]:.2f} seconds")
print(f" CPU time: {t2[1] - t1[1]:.2f} seconds")

In [ ]:
# Apply most basic quality cuts, taken from Ruben work
size_outside = 500 # pe
df_good_data = all_data_df[(all_data_df['ring_completeness'] > 0.9) & (all_data_df['size_outside'] < size_outside) & (all_data_df['ring_width'] < 0.5) & all_data_df['good_ring']]
df_good_data = all_data_df

In [ ]:
# Step 1: Create bins for 'ring_width'
df_good_data['ring_width_bin'] = pd.cut(df_good_data['ring_width'], bins=300)

# Step 2: Group by the new bin column and calculate mean muon efficiency for each bin
binned_data = df_good_data.groupby('ring_width_bin')['muon_efficiency'].mean().reset_index()

# Step 3: Convert the bin intervals to strings (for plotting) or use midpoints
binned_data['bin_mid'] = binned_data['ring_width_bin'].apply(lambda x: x.mid)

# Plotting
plt.figure(figsize=(10, 6))

# Scatter plot of binned values
plt.scatter(binned_data['bin_mid'], binned_data['muon_efficiency'], label='Binned Muon efficiency from LST Data')

# Calculate and plot the regression line over the original data
m, b = np.polyfit(df_good_data['ring_width'], df_good_data['muon_efficiency'], 1)
plt.plot(df_good_data['ring_width'], m*df_good_data['ring_width'] + b, color='red', label='Regression Line', alpha = 0.75)

plt.xlabel('Ring Width [deg]')
plt.ylabel('Optical Efficiency')
plt.legend()
plt.grid(alpha=0.5)
plt.show()
#plt.savefig('/home/jovyan/OptEffVSWidth.png', dpi=300, format='png', bbox_inches='tight')
